In [1]:
#pip install convokit
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('vader_lexicon')

from convokit import Corpus, download
corpus = Corpus(filename=download("movie-corpus"))

#corpus.print_summary_stats()

import pandas as pd
import nltk
from nltk import ne_chunk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from convokit import Corpus, download
from nltk import ne_chunk, pos_tag
from nltk.tokenize import word_tokenize
from convokit import Corpus, download


In [2]:

#making the dataFrame

# Initialize lists to store data
convo_ids = []
movie_indices = []
movie_names = []
release_years = []
ratings = []
votes = []
genres = []
utterance_ids = []
speakers = []
conversation_ids = []
reply_tos = []

texts = []

#Iterate through conversations to extract data
for conv_id in corpus.get_conversation_ids():
    conversation = corpus.get_conversation(conv_id)
    movie_idx = conversation.meta['movie_idx']
    movie_name = conversation.meta['movie_name']
    release_year = conversation.meta['release_year']
    rating = conversation.meta['rating']
    vote = conversation.meta['votes']
    genre = conversation.meta['genre']
   
    
    for utt in conversation.iter_utterances():
        convo_ids.append(conv_id)
        movie_indices.append(movie_idx)
        movie_names.append(movie_name)
        release_years.append(release_year)
        ratings.append(rating)
        votes.append(vote)
        genres.append(genre)
        utterance_ids.append(utt.id)
        speakers.append(utt.speaker.id)
        conversation_ids.append(utt.conversation_id)
        reply_tos.append(utt.reply_to)
       
        texts.append(utt.text)


df = pd.DataFrame({
    'Conversation_ID': convo_ids,
    'Movie_Index': movie_indices,
    'Movie_Name': movie_names,
    'Release_Year': release_years,
    'Rating': ratings,
    'Votes': votes,
    'Genre': genres,
    'Utterance_ID': utterance_ids,
    'Speaker': speakers,
    'Conversation_ID_Utterance': conversation_ids,
    'Reply_To': reply_tos,
    
    'Text': texts
})


df.dropna().drop_duplicates()

df



,Conversation_ID,Movie_Index,Movie_Name,Release_Year,Rating,Votes,Genre,Utterance_ID,Speaker,Conversation_ID_Utterance,Reply_To,Text
0,L1044,m0,10 things i hate about you,1999,6.90,62847,"['comedy', 'romance']",L1045,u0,L1044,L1044,They do not!
1,L1044,m0,10 things i hate about you,1999,6.90,62847,"['comedy', 'romance']",L1044,u2,L1044,None,They do to!
2,L984,m0,10 things i hate about you,1999,6.90,62847,"['comedy', 'romance']",L985,u0,L984,L984,I hope so.
3,L984,m0,10 things i hate about you,1999,6.90,62847,"['comedy', 'romance']",L984,u2,L984,None,She okay?
4,L924,m0,10 things i hate about you,1999,6.90,62847,"['comedy', 'romance']",L925,u0,L924,L924,Let's go.
...,...,...,...,...,...,...,...,...,...,...,...,...
304708,L666369,m616,zulu dawn,1979,6.40,1911,"['action', 'adventure', 'drama', 'history', 'w...",L666371,u9030,L666369,L666370,Lord Chelmsford seems to want me to stay back ...
304709,L666369,m616,zulu dawn,1979,6.40,1911,"['action', 'adventure', 'drama', 'history', 'w...",L666370,u9034,L666369,L666369,I'm to take the Sikali with the main column to...
304710,L666369,m616,zulu dawn,1979,6.40,1911,"['action', 'adventure', 'drama', 'history', 'w...",L666369,u9030,L666369,None,"Your orders, Mr Vereker?"
304711,L666256,m616,zulu dawn,1979,6.40,1911,"['action', 'adventure', 'drama', 'history', 'w...",L666257,u9030,L666256,L666256,"Good ones, yes, Mr Vereker. Gentlemen who can ..."


In [3]:
def recognize_emotion(dialogue):
    # Create a SentimentIntensityAnalyzer instance
    sia = SentimentIntensityAnalyzer()

    # Perform sentiment analysis on the dialogue
    sentiment = sia.polarity_scores(dialogue)

    # Determine emotion based on sentiment scores
    if sentiment['compound'] >= 0.05:
        emotion = 'positive'
    elif sentiment['compound'] <= -0.05:
        emotion = 'negative'
    else:
        emotion = 'neutral'

    return emotion

def add_sentiment_column(df):
    # Create a SentimentIntensityAnalyzer instance
    sia = SentimentIntensityAnalyzer()

    # Create an empty list to store emotions
    emotions = []

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Perform sentiment analysis on the dialogue
        sentiment = sia.polarity_scores(row['Text'])

        # Determine emotion based on sentiment scores
        if sentiment['compound'] >= 0.05:
            emotion = 'positive'
        elif sentiment['compound'] <= -0.05:
            emotion = 'negative'
        else:
            emotion = 'neutral'

        # Store emotion in the list
        emotions.append(emotion)

    # Add a new column to the DataFrame with emotions
    df['Emotion'] = emotions

# Example usage
# Assuming 'YourMovieTitle' is the movie you want to analyze
movie_to_analyze = df[df['Movie_Name'] == 'YourMovieTitle']

# Add a hypothetical 'Timestamp' column representing the temporal aspect
movie_to_analyze['Timestamp'] = range(1, len(movie_to_analyze) + 1)

# Add emotion column to the entire DataFrame
add_sentiment_column(df)



In [4]:
df.to_json('df.json', orient='records', lines=True)

In [18]:

dfs = "C:\\Users\\DELL\\Desktop\\VisProject\\df.csv"


# Save the DataFrame to a new CSV file
df.to_csv(dfs, sep=',', index=False, encoding='utf-8')

In [20]:
# Prompt the user for a dialogue
user_dialogue = input("Enter a dialogue: ")

# Check if the dialogue exists in the DataFrame
result = df[df['Text'] == user_dialogue]

if not result.empty:
    # Extract relevant information
    movie_name = result['Movie_Name'].values[0]
    rating = result['Rating'].values[0]
    emotion = result['Emotion'].values[0]

    print(f"Movie: {movie_name}")
    print(f"Rating: {rating}")
    print(f"Emotion of the dialogue is: {emotion}")
else:
    print("Dialogue not found in the dataset.")


Enter a dialogue: Your orders, Mr Vereker?
Movie: zulu dawn
Rating: 6.40
Emotion of the dialogue is: neutral


In [3]:

#new character dataset for the entire dataset
import pandas as pd
import spacy

# Load spaCy model for NER
nlp = spacy.load("en_core_web_sm")

# Assuming your movie DataFrame is named df
# ...

# Function to perform NER on the entire dataset
def perform_ner_on_dataset(df):
    # Create dictionaries to store character frequencies along with speaker, reply_to, and dialogue
    character_counts = {}
    speaker_dict = {}
    reply_to_dict = {}
    dialogue_dict = {}

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Process the dialogue using spaCy
        doc = nlp(row['Text'])

        # Extract named entities (characters) from the dialogue
        characters = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
       
        # Count the occurrences of each character
        for character in characters:
            if character in character_counts:
                character_counts[character] += 1
            else:
                character_counts[character] = 1

            # Store speaker, reply_to, and dialogue information
            speaker_dict[character] = row['Speaker']
            reply_to_dict[character] = row['Reply_To']
            dialogue_dict[character] = row["Text"]

    # Create a DataFrame from the dictionaries
    df_result = pd.DataFrame(list(character_counts.items()), columns=['Character', 'Frequency'])

    # Sort characters based on frequency in descending order
    df_result = df_result.sort_values(by='Frequency', ascending=False)

    # Add columns for movie name, speaker, reply_to, and dialogue
    df_result['Movie_Name'] = df['Movie_Name'].iloc[0]
    df_result['Speaker'] = df_result['Character'].map(speaker_dict)
    df_result['Reply_To'] = df_result['Character'].map(reply_to_dict)
    df_result['Dialogue'] = df_result['Character'].map(dialogue_dict)

    return df_result

# Create a list to store DataFrames for each movie
dfs = []

# Iterate through unique movie names in the DataFrame
for movie_name in df['Movie_Name'].unique():
    # Perform NER on dialogues of each movie
    character_frequencies_movie = perform_ner_on_dataset(df[df['Movie_Name'] == movie_name])
    
    # Append the result to the list
    dfs.append(character_frequencies_movie)

# Concatenate the DataFrames in the list
df_final_result = pd.concat(dfs, ignore_index=True)

# Print the resulting DataFrame
print(df_final_result)


             Character  Frequency                  Movie_Name Speaker  \
0                 Joey          8  10 things i hate about you      u7   
1              Cameron          7  10 things i hate about you      u7   
2              Patrick          5  10 things i hate about you      u8   
3              William          4  10 things i hate about you      u6   
4       Sarah Lawrence          4  10 things i hate about you     u11   
...                ...        ...                         ...     ...   
23079            Noggs          1                   zulu dawn   u9027   
23080           Bugler          1                   zulu dawn   u9025   
23081              Boy          1                   zulu dawn   u9025   
23082             Cook          1                   zulu dawn   u9026   
23083  William Vereker          1                   zulu dawn   u9034   

      Reply_To                                           Dialogue  
0         L381  Patrick, Pat, you're not looking at the

In [28]:
freqchar = "C:\\Users\\DELL\\Desktop\\VisProject\\freqchar.csv"


# Save the DataFrame to a new CSV file
df_final_result.to_csv(freqchar, sep=',', index=False, encoding='utf-8')

In [26]:
df_final_result

,Character,Frequency,Movie_Name,Speaker,Reply_To,Dialogue,Sentiment
0,Joey,8,10 things i hate about you,u7,L381,"Patrick, Pat, you're not looking at the big pi...",0.100000
1,Cameron,7,10 things i hate about you,u7,L375,"The situation is, my man Cameron here has a ma...",0.062500
2,Patrick,5,10 things i hate about you,u8,None,"I don't understand, Patrick. You haven't done...",0.000000
3,William,4,10 things i hate about you,u6,L151,You could always go with me. I'm sure William...,0.500000
4,Sarah Lawrence,4,10 things i hate about you,u11,L170,Sarah Lawrence is on the other side of the cou...,-0.125000
...,...,...,...,...,...,...,...
23079,Noggs,1,zulu dawn,u9027,None,"What o'clock is it, Mr Noggs?",0.000000
23080,Bugler,1,zulu dawn,u9025,L666361,"I listened extra careful to your ""Stand To"" th...",0.058333
23081,Boy,1,zulu dawn,u9025,L666361,"I listened extra careful to your ""Stand To"" th...",0.058333
23082,Cook,1,zulu dawn,u9026,L666362,"From the Cook, Sir They saw me dip your shavin...",-0.016667


In [25]:

from textblob import TextBlob

# Add a new column for sentiment analysis using TextBlob
df_final_result['Sentiment'] = df_final_result['Dialogue'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Group by character and calculate average sentiment
character_sentiments = df.groupby('Character')['Sentiment'].mean()

# Print the aggregated emotions for each character
print(character_sentiments)

KeyError: 'Character'

In [37]:
#new counts 
import pandas as pd
import spacy
from collections import Counter

# Load spaCy model for NLP
nlp = spacy.load("en_core_web_sm")

# Assuming your movie DataFrame is named df
# ...

# Function to perform NLP analysis on a specific movie from the dataset
def analyze_movie(df, movie_name):
    # Filter the DataFrame to get dialogues of the specified movie
    movie_df = df[df['Movie_Name'] == movie_name]

    # Concatenate all dialogues into a single text for the movie
    movie_text = ' '.join(movie_df['Text'])

    # Process the concatenated text using spaCy
    doc = nlp(movie_text)

    # Extract words from the text, excluding stop words, articles, and one-character words
    words = [token.text.lower() for token in doc if token.is_alpha and
             token.text.lower() not in nlp.Defaults.stop_words and
             token.pos_ != 'DET' and len(token.text) > 1]

    # Count the occurrences of each word
    word_counts = Counter(words)

    # Filter words based on the threshold (more than 5 occurrences)
    filtered_words = {word: count for word, count in word_counts.items() if count > 5}

    # Create a list to store word-dialogue pairs with additional information
    word_dialogue_list = []

    # Iterate through filtered words and find dialogues
    for word in filtered_words.keys():
        dialogues = [dialogue for dialogue in movie_df[movie_df['Text'].str.lower().str.contains(word)]['Text']]
        for dialogue in dialogues:
            # Extract additional information from the corresponding row in movie_df
            row = movie_df[movie_df['Text'] == dialogue].iloc[0]
            word_dialogue_list.append({
                'Word': word,
                'Dialogue': dialogue,
                'Movie_Name': row['Movie_Name'],
                'Rating': row['Rating'],
                'Speaker': row['Speaker'],
                'Replied_To': row['Reply_To']
            })

    # Create a DataFrame from the list
    df_word_dialogues = pd.DataFrame(word_dialogue_list)

    return df_word_dialogues

# Get the name and rating of the first movie in the dataset
first_movie_name = df['Movie_Name'].iloc[0]
first_movie_rating = df['Rating'].iloc[0]

# Analyze dialogues of the first movie
df_word_dialogues_first_movie = analyze_movie(df, first_movie_name)

# Print the resulting DataFrame
df_word_dialogues_first_movie


,Word,Dialogue,Movie_Name,Rating,Speaker,Replied_To
0,okay,She okay?,10 things i hate about you,6.90,u2,None
1,okay,Okay -- you're gonna need to learn how to lie.,10 things i hate about you,6.90,u0,L871
2,okay,Okay... then how 'bout we try out some French ...,10 things i hate about you,6.90,u2,L196
3,okay,"Well, I thought we'd start with pronunciation,...",10 things i hate about you,6.90,u2,L194
4,okay,Okay! I wasn't sure,10 things i hate about you,6.90,u2,L424
...,...,...,...,...,...,...
888,big,I know. It'd have to be a pretty big deal to ...,10 things i hate about you,6.90,u9,L642
889,big,Leave it to you to use big words when you're s...,10 things i hate about you,6.90,u9,L626
890,big,"You're not a big talker, are you?",10 things i hate about you,6.90,u9,None
891,big,Yeah. She left with some bikers Big ones. Fu...,10 things i hate about you,6.90,u5,L986


In [10]:
import pandas as pd
import spacy
from collections import Counter

# Load spaCy model for NLP
nlp = spacy.load("en_core_web_sm")

# Assuming your movie DataFrame is named df
# ...

# Function to perform NLP analysis on the entire dataset
def analyze_dataset(df):
    # Create a list to store word-dialogue pairs with additional information
    word_dialogue_list = []

    # Iterate through each movie in the dataset
    for movie_name in df['Movie_Name'].unique():
        # Filter the DataFrame to get dialogues of the specified movie
        movie_df = df[df['Movie_Name'] == movie_name]

        # Concatenate all dialogues into a single text for the movie
        movie_text = ' '.join(movie_df['Text'])

        # Process the concatenated text using spaCy
        doc = nlp(movie_text)

        # Extract words from the text, excluding stop words, articles, and one-character words
        words = [token.text.lower() for token in doc if token.is_alpha and
                 token.text.lower() not in nlp.Defaults.stop_words and
                 token.pos_ != 'DET' and len(token.text) > 1]

        # Count the occurrences of each word
        word_counts = Counter(words)

        # Filter words based on the threshold (more than 5 occurrences)
        filtered_words = {word: count for word, count in word_counts.items() if count > 5}

        # Iterate through filtered words and find dialogues
        for word, frequency in filtered_words.items():
            dialogues = [dialogue for dialogue in movie_df[movie_df['Text'].str.lower().str.contains(word)]['Text']]
            for dialogue in dialogues:
                # Extract additional information from the corresponding row in movie_df
                row = movie_df[movie_df['Text'] == dialogue].iloc[0]
                word_dialogue_list.append({
                    'Word': word,
                    'Dialogue': dialogue,
                    'Movie_Name': row['Movie_Name'],
                    'Rating': row['Rating'],
                    'Speaker': row['Speaker'],
                    'Replied_To': row['Reply_To'],
                    'Frequency': frequency
                })

    # Create a DataFrame from the list
    df_word_dialogues = pd.DataFrame(word_dialogue_list)

    return df_word_dialogues

# Analyze dialogues for the entire dataset
df_word_dialogues_entire_dataset = analyze_dataset(df)

# Print the resulting DataFrame
print(df_word_dialogues_entire_dataset)


        Word                                           Dialogue  \
0       okay                                          She okay?   
1       okay     Okay -- you're gonna need to learn how to lie.   
2       okay  Okay... then how 'bout we try out some French ...   
3       okay  Well, I thought we'd start with pronunciation,...   
4       okay                               Okay!  I wasn't sure   
...      ...                                                ...   
570145  lord  Excuse me, My Lord.  Norris-Newman, of "The St...   
570146  lord  My Lord.  This list was prepared for you. I do...   
570147  lord  Sikali Horse, My Lord. Christians alL I know e...   
570148  lord  Um. There are rumours that my Lord Chelmsford ...   
570149  lord  Lord Chelmsford seems to want me to stay back ...   

                        Movie_Name Rating Speaker Replied_To  Frequency  
0       10 things i hate about you   6.90      u2       None         10  
1       10 things i hate about you   6.90      

In [11]:
df_word_dialogues_entire_dataset = df_word_dialogues_entire_dataset.drop_duplicates(subset="Word")

# Extract only the "word" and "freq" columns
df_word_dialogues_entire_dataset = df_word_dialogues_entire_dataset[["Word", "Frequency","Movie_Name"]]

df_word_dialogues_entire_dataset

,Word,Frequency,Movie_Name
0,okay,10,10 things i hate about you
9,let,13,10 things i hate about you
31,need,12,10 things i hate about you
44,know,43,10 things i hate about you
88,like,43,10 things i hate about you
...,...,...,...
569545,mutants,8,x-men
569755,fronkonsteen,7,young frankenstein
569861,mmmmm,10,young frankenstein
569974,delbruck,6,young frankenstein


In [12]:
df_word_dialogues_entire_dataset.to_json('wordfreq.json', orient='records', lines=True)

In [13]:
wf = "C:\\Users\\DELL\\Desktop\\VisProject\\wordfreq.csv"


# Save the DataFrame to a new CSV file
df_word_dialogues_entire_dataset.to_csv(wf, sep=',', index=False, encoding='utf-8')

KeyError: 'character'

In [23]:
pip install textblob


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
